## Loading dataset

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
import warnings
warnings.simplefilter('ignore')

import pickle
datasets = pickle.load(open("../data/fs-patch/fs_datasets.pickle", 'rb'))
fss = ['ext3', 'ext4', 'btrfs', 'xfs', 'jfs', 'reiserfs']

import spacy
nlp = spacy.load('en')


## Training classifiers

In [ ]:
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics
from sklearn.preprocessing import StandardScaler, label_binarize, binarize, Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC
import functools
import nltk
from nltk.stem.porter import PorterStemmer
from scipy.stats import norm
from scipy.sparse import csr_matrix

stemmer = PorterStemmer()
rf = functools.partial(sklearn.ensemble.RandomForestClassifier, n_estimators=300)

def binary_bug(dp):
    if dp['type'] == 'b':
        return 1
    else:
        return 0
    
def multi_patch_type(dp):
    return dp['type']

def multi_bug_cons(dp):
    return dp['cons_type']

def only_bug_filter(dp):
    return dp['type'] == 'b'
        
def lemmatize(text):
    lemmatized = []
    doc = nlp(text)
    for token in doc:
        if token.pos_ != 'PUNCT':
            lemmatized.append(token.lemma_)
    return lemmatized

def stem(text):
    stemmed = []
    tokens = nltk.word_tokenize(text)
    for token in tokens:
        stemmed.append(stemmer.stem(token))
    return stemmed

def bns(tprs, fprs):
    """
    Args:
        tprs: A row vector of shape (1, num_features)
        fprs: A row vector of shape (1, num_features)
    """
    num_features = tprs.shape[1]
    bns = np.zeros(num_features)
    for i in range(num_features):
        bns[i] = np.abs(norm.ppf(tprs[0, i]) - norm.ppf(fprs[0, i]))
    return bns

def tfbns(tfs, bns):
    """
    Args:
        tfs: A sparse matrix of shape (num_samples, num_features)
            in csr format
        bns: A numpy array of shape (num_features,)
    """
    cx = tfs.tocoo()
    data, rows, cols = [], [], []
    for i, j, v in zip(cx.row, cx.col, cx.data):
        data.append(v * bns[j])
        rows.append(i)
        cols.append(j)
    return csr_matrix((data, (rows, cols)), shape=tfs.shape)

class Experiment():
    
    def __init__(self, datasets, fss):
        """
        Args:
            datasets: A dictionary, keys are dataset names, each dataset
                is a list of data points (also dictionaries).
            fss: A list of file system names.
        """
        self.datasets = datasets
        self.fss = fss

    def _clean(self):
        self.train_texts = {}
        self.train_extras = {}
        self.train_targets = {}
        self.train_tfidfs = {}
        self.train_vectors = {}
        self.train_counts = {}
        self.train_tfs = {}
        self.train_tfbns = {}
        
        self.test_texts = {}
        self.test_extras = {}
        self.test_targets = {}
        self.test_tfidfs = {}
        self.test_vectors = {}
        self.test_counts = {}
        self.test_tfs = {}
        self.test_tfbns = {}

        self.vectorizers = {}
        self.tf_transformers = {}
        self.feature_labels = {}
        self.classifiers = {}

    def leave_one_dataset_out(self, label_func, init_clf, ngram_range=(1, 1), 
                              text_feature='message', extra_features=None, 
                              dp_filter=lambda dp: True, tokenizer=None, 
                              max_features=None, min_df=1):
        """Perform experiment in a leave-one-out style
        
        Args:
            label_func: A function, takes a data point as input and
                return its target label.
            init_clf: A function, return a classifier which supports
                'fit' and 'score' method
            text_feature: A string, can be either 'message' or 'subject'.
                If set to None, then texts will not be used.
            extra_features: A list of strings.
            dp_filter: A function decides which data point to exclude.
            tokenizer: A function takes a string and return a list of tokens.
            max_features: An int or None. If not None, only consider
                top max_features ordered by term frequency across the corpus.
            min_df: An int, ignore terms when building vocabulary if their
                document frequency is strictly lower than this threshold.
        """
        
        def gather_data(fs, text_list, target_list, extra_list):
            """Helper function to gather data from a single dataset"""
            for dp in self.datasets[fs]:
                if dp_filter(dp):
                    if extra_features and 'message' in dp:
                        if text_feature:
                            text_list.append(dp[text_feature])
                        target_list.append(label_func(dp))
                        extra_list.append([dp[f] for f in extra_features])
                    elif not extra_features and text_feature in dp:
                        text_list.append(dp[text_feature])
                        target_list.append(label_func(dp))
                
        assert((text_feature and text_feature in ('message', 'subject')) or extra_features)
        self._clean()

        for fs in self.fss:

            self.train_texts[fs], self.train_targets[fs], \
                self.train_extras[fs] = [], [], []
            self.test_texts[fs], self.test_targets[fs], \
                self.test_extras[fs] = [], [], []

            for fs2 in self.fss:
                if fs2 != fs:
                    gather_data(fs2, self.train_texts[fs],
                        self.train_targets[fs], self.train_extras[fs])

            gather_data(fs, self.test_texts[fs], self.test_targets[fs],
                self.test_extras[fs])

            self.feature_labels[fs] = []

            if text_feature:
                # vectorize texts
                self.vectorizers[fs] = CountVectorizer(
                                            tokenizer=tokenizer,
                                            ngram_range=ngram_range,
                                            max_features=max_features,
                                            min_df=min_df)
                
                self.train_counts[fs] = \
                    self.vectorizers[fs].fit_transform(self.train_texts[fs])
                self.test_counts[fs] = \
                    self.vectorizers[fs].transform(self.test_texts[fs])
    
                self.tf_transformers[fs] = TfidfTransformer(use_idf=False, norm=None)
                
                self.train_tfs[fs] = \
                    self.tf_transformers[fs].fit_transform(self.train_counts[fs])
                self.test_tfs[fs] = \
                    self.tf_transformers[fs].transform(self.test_counts[fs])
                    
                binary_train_counts = binarize(self.train_counts[fs])
                    
                classes = np.unique(self.train_targets[fs])
                # label_binarize return column vector of shape (num_samples, 1)
                binary_train_targets = label_binarize(self.train_targets[fs], classes)
                
                train_pos = np.sum(binary_train_targets)
                train_neg = binary_train_targets.shape[0] - train_pos
                
                # train_tp is a row vector of shape (1, num_features)
                train_tps = np.sum(binary_train_counts[np.nonzero(binary_train_targets)[0]], axis=0)
                train_fps = np.sum(binary_train_counts[np.argwhere(binary_train_targets == 0)[:, 0]], axis=0)
                
                train_tprs = np.clip(train_tps / train_pos, 0.0005, 0.9995)
                train_fprs = np.clip(train_fps / train_neg, 0.0005, 0.9995)
                train_bns = bns(train_tprs, train_fprs)
                
                self.train_tfbns[fs] = tfbns(self.train_tfs[fs], train_bns)
                self.test_tfbns[fs] = tfbns(self.test_tfs[fs], train_bns)
                
                normalizer = Normalizer()
                self.train_tfbns[fs] = normalizer.transform(self.train_tfbns[fs])
                self.test_tfbns[fs] = normalizer.transform(self.test_tfbns[fs])
            
                """
                self.vectorizers[fs] = TfidfVectorizer(
                                            tokenizer=tokenizer, 
                                            ngram_range=ngram_range, 
                                            max_features=max_features)
                self.train_tfidfs[fs] = \
                    self.vectorizers[fs].fit_transform(self.train_texts[fs])
                self.test_tfidfs[fs] = \
                    self.vectorizers[fs].transform(self.test_texts[fs])
                """

                vocab = self.vectorizers[fs].vocabulary_
                self.feature_labels[fs] = [None] * len(vocab)
                for v in vocab:
                    self.feature_labels[fs][vocab[v]] = v

            if extra_features:
                self.train_extras[fs] = np.array(self.train_extras[fs])
                self.test_extras[fs] = np.array(self.test_extras[fs])

                # scale to have zero mean and unit variance
                scaler = StandardScaler()
                self.train_extras[fs] = scaler.fit_transform(self.train_extras[fs])
                self.test_extras[fs] = scaler.transform(self.test_extras[fs])

                for f in extra_features:
                    self.feature_labels[fs].append(f)
                    
            if text_feature and extra_features:
                self.train_vectors[fs] = \
                    np.hstack((self.train_tfbns[fs].todense(), self.train_extras[fs]))
                self.test_vectors[fs] = \
                    np.hstack((self.test_tfbns[fs].todense(), self.test_extras[fs]))
            elif text_feature and not extra_features:
                self.train_vectors[fs] = self.train_tfbns[fs]
                self.test_vectors[fs] = self.test_tfbns[fs]
            elif not text_feature and extra_features:
                self.train_vectors[fs] = self.train_extras[fs]
                self.test_vectors[fs] = self.test_extras[fs]
        

            self.classifiers[fs] = init_clf()
            self.classifiers[fs].fit(self.train_vectors[fs], self.train_targets[fs])

            print('----- Test Accuracy for %s -----' % fs)
            print('Classifier: %.3f' % self.classifiers[fs].score(
                self.test_vectors[fs], self.test_targets[fs]))

            """
            pred2 = []
            for text in self.test_texts[fs]:
                if 'fix' in text.lower(): 
                    pred2.append(1)
                else:
                    pred2.append(0)
            print('Naive: %.3f' % sklearn.metrics.accuracy_score(
                self.test_targets[fs], pred2))
            """
        
        print('Average number of features: %.1f' % 
              np.mean(np.array([len(self.feature_labels[fs]) for fs in fss])))
            
exp = Experiment(datasets, fss)

## Binary Classification of Bug Fix Patch

In [ ]:
# binary classification of bug patch, only using features derived from code
exp.leave_one_dataset_out(binary_bug, LinearSVC, text_feature=None,
                          extra_features=['num_files', 'num_adds', 'num_dels'])

In [ ]:
# binary classification of bug patch, only using text feature
exp.leave_one_dataset_out(binary_bug, LinearSVC)

In [ ]:
# binary classification of bug patch, only using text feature 
# and drop terms with frequency lower than min_df
exp.leave_one_dataset_out(binary_bug, LinearSVC, min_df=3)

In [ ]:
# binary classification of bug patch, using text feature with features derived from code
exp.leave_one_dataset_out(binary_bug, LinearSVC, 
                          extra_features=['num_files', 'num_adds', 'num_dels'])

In [ ]:
# binary classification of bug patch, using stemmed text feature and setting max_features to 5000
exp.leave_one_dataset_out(binary_bug, LinearSVC, tokenizer=stem, max_features=5000,
                         extra_features=['num_files', 'num_adds', 'num_dels'])

In [ ]:
# binary classification of bug patch, using bigram text feature with features derived from code
exp.leave_one_dataset_out(binary_bug, LinearSVC, ngram_range=(1, 2), 
                          extra_features=['num_files', 'num_adds', 'num_dels'])

In [ ]:
# binary classification of bug patch, using lemmatized text feature with features derived from code
exp.leave_one_dataset_out(binary_bug, LinearSVC, tokenizer=lemmatize,
                          extra_features=['num_files', 'num_adds', 'num_dels'])

In [ ]:
# binary classfication of bug patch, using stemmed text feature with features derived from code
exp.leave_one_dataset_out(binary_bug, LinearSVC, tokenizer=stem, 
                          extra_features=['num_files', 'num_adds', 'num_dels'])

## Multi-Class Classification of Patch Type

In [ ]:
# multi-class classification of patch type, only using text feature
exp.leave_one_dataset_out(multi_patch_type, LinearSVC)

In [ ]:
# multi-class classification of patch type, using text feature with features derived from code
exp.leave_one_dataset_out(multi_patch_type, LinearSVC,
                         extra_features=['num_files', 'num_adds', 'num_dels'])

## Multi-Class Classification of Bug Consequences

In [ ]:
# multi-class classification of bug consequences, only using text feature
exp.leave_one_dataset_out(multi_bug_cons, LinearSVC, dp_filter=only_bug_filter)

In [ ]:
# multi-class classification of bug consequences, using bigram text feature
exp.leave_one_dataset_out(multi_bug_cons, LinearSVC, dp_filter=only_bug_filter, ngram_range=(1, 2))

In [ ]:
# multi-class classification of bug consequences, using text feature with features derived from code
exp.leave_one_dataset_out(multi_bug_cons, LinearSVC, dp_filter=only_bug_filter,
                         extra_features=['num_files', 'num_adds', 'num_dels'])

## Explaning Random Forest

In [ ]:
import pprint
pp = pprint.PrettyPrinter()

In [ ]:
# explaning random forest classifier
def rank_features_by_importance(exp, top_n=20):
    if 'feature_importances_' in dir(exp.classifiers['ext3']):
        for fs in fss:
            print('------- important features for %s -------' % fs)
            truncated_importances = map(lambda x: '%.4f' % x, exp.classifiers[fs].feature_importances_)
            pp.pprint(sorted(zip(truncated_importances, exp.feature_labels[fs]), reverse=True)[:top_n])
    else:
        print("classifiers don't have attribute feature_importance_")

In [ ]:
rank_features_by_importance(exp, top_n=20)

## Explaining SVM

In [ ]:
# explaining linear SVM classifier
def rank_features_by_weight(classifiers, classes, feature_labels, 
                            top_n=20, individual_class=False):
    """Rank features by the absolute value of associated primal weight
    
    Args:
        classifiers: A dictionary, with key being the file system name
            and value being the corresponding classifier.
        classes: A list of class labels.
        feature_labels: A dictionary of list of feature names, each key
            is a file system name, features are in the order as classifier
            sees them.
        top_n: An integer, specifies number of top features to print.
        individual_class: A boolean value, whether use sum of absolute weights 
            across classes. In binary tasks, this value doesn't matter.
    """
    
    fss = list(classifiers.keys())
    if 'coef_' in dir(classifiers[fss[0]]):
        for fs in fss:
            print('------- %s -------' % fs)
            if individual_class:
                coef = classifiers[fs].coef_
                for i in range(coef.shape[0]):
                    print('\t------ %s ------' % classes[i])
                    # print('\t------  ------')
                    order = np.argsort(np.absolute(coef[i]))
                    pp.pprint(np.array(feature_labels[fs])[order][-top_n:])
                print()
            else:
                coef = classifiers[fs].coef_
                num_features = coef.shape[1]
                abs_coef_sum = np.zeros(num_features)
                for i in range(coef.shape[0]):
                    abs_coef_sum += np.absolute(coef[i])
                order = np.argsort(abs_coef_sum)
                pp.pprint(np.array(feature_labels[fs])[order][-top_n:])
    else:
        print("classifiers don't have attribute coef_")

In [ ]:
rank_features_by_weight(exp.classifiers, np.unique(exp.train_targets['ext3']), 
                        exp.feature_labels, top_n=20)

In [ ]:
rank_features_by_weight(exp.classifiers, np.unique(exp.train_targets['ext3']), 
                        exp.feature_labels, top_n=20, individual_class=True)

## LIME

In [ ]:
from sklearn.pipeline import make_pipeline
c = make_pipeline(vectorizers['ext3'], classifiers['ext3'])
print(c.predict_proba([test_texts['ext3'][1]]))

In [ ]:
from lime.lime_text import LimeTextExplainer
class_names = ['not-bug', 'bug']
explainer = LimeTextExplainer(class_names=class_names)

In [ ]:
def explain_pred(idx, fs):
    c = make_pipeline(vectorizers[fs], classifiers[fs])
    exp = explainer.explain_instance(test_texts[fs][idx].lower(), c.predict_proba, num_features=8)
    print('Patch id: %d' % idx)
    print('Probability(bug) =', c.predict_proba([test_texts[fs][idx]])[0,1])
    print('True class: %s' % class_names[test_targets[fs][idx]])
    print('Text: %s' % test_texts[fs][idx])
    pp.pprint(exp.as_list())
    # exp.show_in_notebook(text=True)

In [ ]:
from sklearn.pipeline import make_pipeline
fs = 'ext3'
c = make_pipeline(vectorizers[fs], classifiers[fs])
for i in range(len(test_texts[fs])):
    if ('fix' not in test_texts[fs][i].lower() 
        and c.predict_proba([test_texts[fs][i]])[0,1] > 0.5
        and test_targets[fs][i] == 1):
        explain_pred(i, fs)

In [ ]:
# keyword 'fix'
for i in [22, 24]:
    explain_pred(i, 'ext3')

In [ ]:
# when keyword 'fix' is absent
for i in [23, 25]:
    explain_pred(i, 'ext3')

In [ ]:
# interesting case
for i in [5, 26]:
    explain_pred(i, 'ext3')